<a href="https://colab.research.google.com/github/DSIMB/AMBROSIA/blob/main/Ambrosia_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to AMBROSIA: carbohydrate binding residues predictor


# Notebook initialization

In [ ]:
#@title ## Load AMBROSIA Github repository
import os
if os.getcwd() == "/content":
    if os.path.exists("AMBROSIA/.git"):
        %cd AMBROSIA
        !git pull
    else:
        !rm -rf AMBROSIA  # Remove if it's not a valid git repo
        !git clone https://github.com/DSIMB/AMBROSIA.git
        %cd AMBROSIA


#if os.getcwd() == "/content":
#    if os.path.exists("AMBROSIA"):
#        !rm -rf AMBROSIA  # Remove the existing directory
#    !git clone https://github.com/DSIMB/AMBROSIA.git
#    %cd AMBROSIA

In [ ]:
#@title ## Install necessary dependencies
!pip install torch fair-esm ankh plotly py3Dmol matplotlib

In [ ]:
#@title ## Import Necessary libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import esm
import ankh
from plotly import graph_objects as go
from google.colab import drive
from collections import defaultdict
import numpy as np
import py3Dmol
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from urllib.request import urlopen
# drive.mount('/content/drive')


# Read input protein sequence

In [ ]:
#@title ## Upload .fasta file
uploaded = files.upload()

# Access the uploaded file
file_path = list(uploaded.keys())[0]
print(f"Uploaded file: {file_path}")

# Reading the uploaded file
with open(file_path, 'r') as file:
    sequence_name = ""
    input_sequence = ""
    for line in file:
        line = line.strip()
        if line.startswith(">"):
            sequence_name = line[1:]  # Remove '>'
        else:
            input_sequence += line

print("Sequence Name:", sequence_name)
print("Input Sequence:", input_sequence)

#sequence_name = "Anti-sigma-W factor RsiW"
#input_sequence = """
#MGNNTGTILEIKGNKAIVMTNTCDFIAITRMPEMFVGQQVDLNNSAIKSKSNPLKYFAIAGMFVLILCSVLIYQLVKPSAVFAYVDVDINPSLELLIDKKANVIEVKTLNSDADALVKDIRLVNKSLTNAVKIIIKESQNKGFIRPDTKNAVLISASINPGKSISSAVSSEKILDVIVSDLQKTDFSIGAVSIKAEVVKVDPIERSEAVKNNISMGRYKLFEEITESDENIDIEKAKTEGLSKIIEEYETKEQEKTIASVDKDNSYKPVQDNKEILDKPKNSTTKDNPKVADNKKPENNNSQKYSNGNSNSSKSSAVKPNKAEDQFKASRSNSENNSSNNRDQSKNTNKKSSDEKKTLDQGSKPITTDDGTKSLNNKNNNKNNDEKPKNHPAKENKQENGNNNQQKSKEKNKK
#"""

input_sequence = input_sequence.replace('\n', '')
start_index = 1
sequence_labels = [f"{aa}{i}" for i, aa in enumerate(input_sequence, start=start_index)]

# Generate embeddings

In [ ]:
#@title ## Generate ESM-2 embedding

esm2_data = [(sequence_name, input_sequence)]

# ESM-2
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

batch_labels, batch_strs, batch_tokens = batch_converter(esm2_data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33])
esm2_embedding = results["representations"][33][0,1:-1]

In [ ]:
#@title ## Generate Ankh embedding

model, tokenizer = ankh.load_large_model()
model.eval()

ankh_data = [list(input_sequence)]
outputs = tokenizer.batch_encode_plus(ankh_data,
                                    add_special_tokens=True,
                                    padding=True,
                                    is_split_into_words=True,
                                    return_tensors="pt")
with torch.no_grad():
    embeddings = model(input_ids=outputs['input_ids'], attention_mask=outputs['attention_mask'])
    ankh_embedding = embeddings.last_hidden_state[0, :-1]

In [ ]:
#@title ## Check resulting embedding dimensions

print(esm2_embedding.shape, len(input_sequence), ankh_embedding.shape)
assert esm2_embedding.shape[0] == len(input_sequence) == ankh_embedding.shape[0], "Something went wrong during embedding generation"


# Perform predictions

In [ ]:
#@title ## Upload model architecture & parameter paths
from model.architecture import CNN

# Define embedding sizes
embedding_sizes = {
    "esm2": esm2_embedding.shape[1] if esm2_embedding.shape[1] else 1280,
    "ankh": ankh_embedding.shape[1] if ankh_embedding.shape[1] else 1536 # 1560
}

# Initialize models dictionary
models = defaultdict(dict)
results = defaultdict(dict)
# Define paths to model parameters
param_paths = {
    "ankh": "./model/weights/ankh/lr_5e-7_bs_64_modelconv_n512_d0.1_ks31_{}.pt",
    "esm2": "./model/weights/esm2/lr_5e-7_bs_64_modelconv_n512_d0.1_ks31_{}.pt",
    #"ankh": "/content/drive/MyDrive/ambrosia_models/models_yangfan/ambrosia_ankh_fold{}.pt",
    #"esm2": "/content/drive/MyDrive/ambrosia_models/models_yangfan/ambrosia_esm2_fold{}.pt"
}

In [ ]:
#@title ## Upload model weights and perform predictions for each embedding
# Load models and evaluate
for model_type in ["ankh", "esm2"]:
    for fold in range(5):
        model = CNN(in_channels=embedding_sizes[model_type], kernel_size=31)
        param_path = param_paths[model_type].format(fold)
        model.load_state_dict(torch.load(param_path, map_location='cpu'))
        model.eval()
        models[model_type][fold] = model

# Perform inference and store results
with torch.no_grad():
    for model_type in ["ankh", "esm2"]:
        embedding = ankh_embedding if model_type == "ankh" else esm2_embedding
        for fold in range(5):
            model = models[model_type][fold]
            logits = model(embedding.transpose(0, 1))
            results[model_type][fold] = logits

In [ ]:
#@title ## Calculate prediction average
from model.calculate_and_plot_proba import calculate_probabilities_and_classes, plot_probabilities

all_probs, ankh_probs, esm2_probs = calculate_probabilities_and_classes(results)

# Calculate individual fold probabilities
fold_probs_esm2 = [torch.sigmoid(torch.tensor(results["esm2"][fold].numpy())).numpy() for fold in range(5)]
fold_probs_ankh = [torch.sigmoid(torch.tensor(results["ankh"][fold].numpy())).numpy() for fold in range(5)]

# Plot predictions

In [ ]:
# Plot all probabilities
plot_probabilities(sequence_labels, all_probs, esm2_probs, ankh_probs, fold_probs_esm2, fold_probs_ankh)

# [Experimental] View on structure



In [ ]:
# Function to read PDB file from URL
def fetch_pdb_from_url(url):
    response = urlopen(url)
    pdb_data = response.read().decode('utf-8').splitlines()
    return pdb_data

# Prompt user for input method
input_method = input("Enter 'upload' to upload a PDB file or 'url' to provide a URL: ").strip().lower()

if input_method == 'upload':
    uploaded = files.upload()
    pdb_filename = next(iter(uploaded))
    with open(pdb_filename, 'r') as file:
        pdb_data = file.readlines()
elif input_method == 'url':
    pdb_url = input("Enter the URL of the PDB file: ").strip()
    pdb_data = fetch_pdb_from_url(pdb_url)
else:
    raise ValueError("Invalid input method. Enter 'upload' or 'url'.")
# Modify B-factors in the PDB file
new_pdb_data = []
prob_index = 0
last_res_id = None
for line in pdb_data:
    if line.startswith("ATOM"):
        res_id = line[22:26].strip()
        if res_id != last_res_id:
            if prob_index < len(all_probs):
                new_line = line[:60] + f"{all_probs[prob_index]*100:6.2f}" + line[66:]
                prob_index += 1
            else:
                new_line = line
            last_res_id = res_id
        else:
            new_line = line[:60] + f"{all_probs[prob_index-1]*100:6.2f}" + line[66:]
        new_pdb_data.append(new_line)
    else:
        new_pdb_data.append(line)

# Save the modified structure to ensure line breaks are correct
modified_pdb_filename = "modified_structure.pdb"
with open(modified_pdb_filename, 'w') as file:
    file.writelines('\n'.join(new_pdb_data))

In [ ]:
# Lire le fichier PDB modifié
with open(modified_pdb_filename, 'r') as file:
    pdb_content = file.read()

# Visualiser la structure avec Py3Dmol
view = py3Dmol.view(width=800, height=600)
view.addModel(pdb_content, "pdb")
view.setStyle({'cartoon': {'color': 'grey'}})

# Colorier les résidus en fonction des probabilités
for i, prob in enumerate(all_probs):
    if prob > 0.5:
      resi = str(i + 1)
      view.addStyle({'resi': resi}, {'stick': {'color': 'red'}})

view.zoomTo()
view.show()